# 深入理解单步物体检测器SSD,YOLO,FPN&RetinaNet
翻译自：https://medium.com/@jonathan_hui/what-do-we-learn-from-single-shot-object-detectors-ssd-yolo-fpn-focal-loss-3888677c5f4d

在第二部分，我们将就单步物体检测器SSD和YOLO（YOLOv2和YOLOv3）展开讨论。我们还将深入FPN来讨论多尺度特征图金字塔怎么提升准确率的，尤其对于单步检测器上表现较差的小物体。之后我们将讨论Focal loss和RetinaNet在训练时如何处理类别不平衡问题。<br>

- 第一部分：理解基于区域的物体检测模型工作原理（Faster R-CNN, R-FCN, FPN）
- 第二部分：理解单次检测模型工作原理（SSD,YOLO）,FPN以及Focal loss
- 第三部分：设计建议和经验，物体检测的发展方向

## 单步检测器
Faster R-CNN包含一个专用的目标提取网络（RPN），在其后是一个分类器。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*F-WbcUMpWSE1tdKRgew2Ug.png)
基于区域的检测器准确性是建立较大的计算消耗之上的。Faster R-CNN在PASCAL VOC 2007测试集上的处理能力是7FPS（每秒7帧）。受R-FCN的启发，研究人员尝试通过减少每个ROI中的工作量来提升性能表现。<br>
```python
feature_maps = process(image)
ROIs = region_proposal(feature_maps)
for ROI in ROIs
    patch = roi_align(feature_maps, ROI)
    results = detector2(patch)    # Reduce the amount of work here!
```
我们一定需要一个单独的目标提取步骤吗？能从特征图通过一步就得到目标框以及分类结果吗？
```python
feature_maps = process(image)
results = detector3(feature_maps) # No more separate step for ROIs
```
让我们重新回到滑动窗口检测器。我们可以在特征图上应用滑动窗口来检测物体。对于不同的物体，我们使用不同形状的窗口。之前滑动窗口的主要问题是我们使用窗口作为最终的目标框。为此，我们需要尽可能多的窗口来覆盖更多的物体。更有效的一种方式是将窗口作为一个初始假设，然后在当前窗口上应用一个检测器同时预测类别和目标框。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*tE6DUwv6VIHu1KlwYmSBTw.jpeg)
这个概念跟Faster R-CNN中的anchors很类似。然而，单步检测器做到了同时预测目标框和类别。这里我们做一个简单的回顾。例如，我们有一个8x8的特征图，每个位置上需要做k个预测，也就是说，一共有8x8xk个预测结果。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*i2egSyxtuJo3YYjdLbaBGQ.png)
在每个位置，我们有k个anchors(anchors就是假设的固定边界框)，每个anchor对应一个预测。我们仔细挑选anchors并且每个位置上使用相同形状的anchor。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*1F8rWQyBV-P8pDn0Avx-OA.png)
这里4个anchors(绿色的框)和4个预测（蓝色的框）都和同一个位置相关联。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*TjZ-YFE1YLPNOJJzjyFCEQ.jpeg)
在Faster R-CNN中，我们使用一个卷积核来做预测，预测结果有5个值：4个是预测的边界框坐标，1个是对象性置信得分。所以3x3xDx5的卷积核将8x8xD特征图转换为8x8x5。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*yrAA9xnL4OlhX6RoeHQVtQ.png)
在单步检测器中，卷积核同样会预测C个类别的分类概率。所以我们应用一个3x3xDx25的卷积核并且将特征图从8x8xD转换为8x8x25，因为C=20。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*UsqjfoW3sLkmyXKQ0Hyo8A.png)
单步检测器通常通过牺牲准确率来得到更好的实时处理速度。他们在处理太近或者太小的物体上还是会有问题。如下图所示，在左下角有9个圣诞老人，但是其中一个单步检测器只检测出了5个。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*j4PnWfxP3yoVPOFyI27tww.jpeg)

## SSD
SSD是一个使用了VGG19网络作为特征提取器（跟Faster R-CNN中的CNN等价）的单步检测器。然后我们在其后加入自定义的卷积层（蓝色），并且使用卷积核（绿色）来做预测。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*1C5hgYTdBvCdCYWbXEaVww.png)
然而，卷积层减少了空间上的维度和解析度。所以上述模型只能检测较大的物体。为了改进，我们在多个特征图上进行独立的物体检测。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*k0eFZw1jlF9xPvhzBKt6LQ.png)
下图可以展示不同特征图上的维度信息。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*up-gIJ9rPkHXUGRoqWuULQ.jpeg)
SSD使用经过多次卷积层之后的卷积层来检测物体。如果我们将上图重新绘制，会发现空间解析度会大大地降低，因此会增大错过较小物体的概率，因为这些物体在低解析度下很难检测。如果有以下问题，我们需要增加输入图像的解析度。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*oCaj3OQIbhOGSxcgvONTQw.png)

## YOLO
YOLO是另一个单步检测器。YOLO使用DarkNet后跟卷积层来做检测。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*NBnDpz8fitkhcdnkgF2bvg.png)
然而，它它不适用多尺度的特征图进行独立的检测。相反，它将特征图压缩并且跟较低解析度上的特征图合并。例如，YOLO将一个28x28x512的层转换为14x14x2048.然后将其和一个14x14x1024的特征图合并。之后，YOLO在这个14x14x3072的特征图上应用卷积核做预测。<br>
YOLO(v2)在实现上做了一些改进，使得mAP从63.4提升到了第一个发行版时的78.6。YOLO9000可以检测9000种不同类别的物体。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*3IdCKSzR5R0lIE1LSmN4Bg.png)
这里YOLO论文中提到的不同检测器在mAP和FPS上的对比。YOLOv2可以接收不同解析度的图像。较低解析度的图像可以达到更高的FPS但是mAP较低。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*NJj17Z6FgffYaA4WH2WIjw.png)

## YOLOv3
YOLOv3使用了更加复杂的骨架网络进行特征提取。Darknet-53主要包含的是3x3和1x1的卷积核，同时还有类似于ResNet中的残差网络结构。Darknet-53相较于ResNet-152有较低的BFLOP（十亿级浮点运算数），但是达到同样准确率的速度要快一倍。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*biRYJyCSv-UTbTQTa4Afqg.png)
YOLOv3使用了特征金字塔来检测较小物体。下图是不同检测器在速度和准确率上的博弈。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*rfj_5yjKZm2LJvVzMXmLFA.png)

## 特征金字塔网络（FPN）
在多尺度上检测物体尤其是较小物体是很有挑战的。FPN是一个基于特征金字塔概念设计的特征提取器用于提升准确率和速度。它可以替换检测器如Faster R-CNN中的特征提取器从而获得更高质量的特征金字塔。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*aMRoAN7CtD1gdzTaZIT5gA.png)
FPN包含一个从底向上和一个从顶向下的路径。从底向上的路径是一个常见的利用卷积网络提取特征的过程。向上的过程中，空间解析度减小。在检测到了更高维度的结构后，每一层的语义信息得以增加。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*_kxgFskpRJ6bsxEjh9CH6g.jpeg)
SSD根据多个特征图做检测。然而，底层的特征图并不用于物体检测，这些层的解析度较高但是语义信息较少，而且会拉慢处理速度。所以SSD只使用上层的特征图做检测，因此在较小的物体上比较较差。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*M_c6Jx5Uy7qr6vJbrtAvhg.png)
FPN提供了一个从定向下的路径将一个语义较丰富的层构建为一个解析度较高的层。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*XmNDHT8WWZbXACyBjg3ZeQ.jpeg)
由于重新构建的层拥有较强的语义信息但是经过降采样和升采样，物体的定位不是很准确。我们将重构层和相关的特征图侧连起来已获得更精确的定位。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*aMRoAN7CtD1gdzTaZIT5gA.png)
下图是一个自底向上和自顶向下路径的详细信息图。P2,P3,P4,P5是用于物体检测的特征图金字塔。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*ffxP_rL8-jMvipLhMJrVeA.png)



## FPN with RPN
FPN本身并不是一个物体检测器。它是一个集成在物体检测器中的特征检测器。特征金字塔中的每一个特征图需要单独进行物体检测。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*cHR4YRqdPBOx4IDqzU-GwQ.png)

## FPN with Fast R-CNN or Faster R-CNN
在FPN中，我们生成了一个特征图金字塔。我们使用RPN来生成ROIs。基于ROI的尺寸，我们可以从FPN生成的特征图中选择最合适的尺度做特征提取。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*Wvn0WG4XZ0w9Ed2fFYPrXw.jpeg)

## 难点挖掘
对于大多数的检测器如SSD和YOLO,我们做出的预测数姚远多于实际存在的物体数。所以会有很多的错误匹配。这会在训练时造成类别失衡。我们训练的模型更多的是在学习背景而不是检测物体。然而，我们需要进行负例采样来研究什么造成了较差的预测。所以，以SSD为例，我们利用置信损失对样本排序，取分数最高的一个，并且保证负例和正例的比例在3:1。这使得训练更加快速和稳定。<br>

## 非极大值抑制（NMS）
检测器对于同一个物体会生成多个检测结果。为了解决这个问题，我们使用了NMS来去除较低置信度的冗余结果。我们将预测结果按照置信度从高到低排序。如果预测结果有同样的类别而且当前位置上IoU大于0.5，就去掉这个结果。

## Focal loss(RetinaNet)
类别失衡会影响性能。SSD通过重新采样物体类别和样本类别，使得结果不被背景影响过多。Focal loss(FL)使用了另一种方法来减小训练较好类别的损失。所以一旦模型在检测背景上表现较好，就会减少它的损失并且重新加强在物体类别上的训练。我们以交叉熵损失CE为例，加入一个权重来减弱高置信度类别的损失。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*DgI0JPk98eXfLvmkK_9PGQ.png)
例如，取$\gamma=0.5$,分类较好的样本上的focal loss将会趋近于0。<br>
![avater](https://cdn-images-1.medium.com/max/800/1*FCV96tP679EScoiwKq4IaQ.png)
这里是一个构建于FPN和ResNet之上的RetinaNet，使用了Focal loss。<br>
![avater](https://cdn-images-1.medium.com/max/1000/1*jQFeF7gj6uCXVzUb08S9lg.png)

## SSD,YOLO&FPN拓展阅读
SSD和YOLO比本文描述的要更加复杂。如果需要进一步的研究，请参考：<br>

- [SSD object detection](https://medium.com/@jonathan_hui/ssd-object-detection-single-shot-multibox-detector-for-real-time-processing-9bd8deac0e06)
- [YOLO object detection](https://medium.com/@jonathan_hui/real-time-object-detection-with-yolo-yolov2-28b1b93e2088)
- [FPN object detection](https://medium.com/@jonathan_hui/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c)